In [3]:
# ✅ 1. 기본 import
import sys
sys.path.append("./전처리")

import os
import importlib
import torch
import merge
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

# ✅ 2. merge.py 실행
# merge.py 가 "전처리/merge.py" 에 있으므로 sys.path에 추가
sys.path.append(os.path.join(os.getcwd(), "전처리"))

import merge  # 전처리 모듈 불러오기

# ✅ 카톡 파일 지정 (변경 가능)
input_chat_path = "datasets/KakaoTalk_20250515_0053_22_930_유정유정.txt"

# ✅ merge 실행 → 정제된 문장 리스트 반환
clean_sentences = merge.run_merge(input_chat_path)  # run_merge() 함수가 merge.py에 있다고 가정

# ✅ 3. 모델 로드
ADAPTER_PATH = "Models/ToneDetect_adapter"
BASE_MODEL_NAME = "beomi/kcbert-base"  # 원래 학습할 때 사용한 베이스

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
base_model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_NAME, num_labels=5)
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()

# ✅ 4. 예측 함수
def predict_style(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_class = torch.argmax(logits, dim=1).item()
    return pred_class

# ✅ 5. 라벨 정의
label_map = {
    0: "chat_emoticon(이모티콘 자주 쓰는 말투)",
    1: "elder_speech(어르신 말투)",
    2: "formal(격식있는 말투)",
    3: "informal(친근한 말투)",
    4: "soft_polite(부드럽고 상냥한 말투)"
}

# ✅ 6. 전체 예측
from collections import defaultdict

style_counts = defaultdict(int)

for sentence in clean_sentences:
    pred = predict_style(sentence)
    style_name = label_map[pred]
    style_counts[style_name] += 1

# ✅ 7. 결과 출력
print("\n📊 예측 결과 요약:")
for style, count in style_counts.items():
    print(f"- {style}: {count} 문장")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



📊 예측 결과 요약:
- elder_speech(어르신 말투): 1 문장
- soft_polite(부드럽고 상냥한 말투): 5 문장
- chat_emoticon(이모티콘 자주 쓰는 말투): 35 문장
- informal(친근한 말투): 7 문장
- formal(격식있는 말투): 4 문장
